# CS4248 Project Group 23

In [1]:
# If you wish to run this on Google Colab, mount the Google drive by running this cell or click the `files` icon on the left navbar
# and click mount Google Drive (it takes some time to load)
# from google.colab import drive
# drive.mount('/content/drive')

# %cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"
# !cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score


In [3]:
# Unzip raw_data.zip locally
import zipfile
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    zip_ref.extractall()

Feature Engineering: Capture various features of the text (e.g. punctuation, stopwords, statement length). 
Test out different tokenizers to capture their performance.


In [4]:
import re
import string
from nltk.tokenize import word_tokenize

# todo parallelize this in future


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

def preprocess_text(text):
    # Lowercase text
    text = text.lower()
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # tokenize
    words = word_tokenize(text)
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # Convert list of words back to string
    text = ' '.join(words)
    return text

In [6]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/t1dus/nltk_data...


True

In [7]:
train = pd.read_csv("./raw_data/fulltrain.csv", names=['Verdict', 'Text'])
test = pd.read_csv("./raw_data/balancedtest.csv", names=['Verdict', 'Text'])

# Preprocess all documents in the corpus
preprocessed_corpus = [preprocess_text(text) for text in train['Text']]

# Fit the TF-IDF vectorizer on the preprocessed corpus
X_train = vectorizer.fit_transform(preprocessed_corpus)
X_test = vectorizer.transform([preprocess_text(text) for text in test['Text']])

y_train = train['Verdict'].apply(lambda x: 1 if x == 4 else 0) # convert to binary- label 4 = trusted
y_test = test['Verdict'].apply(lambda x: 1 if x == 4 else 0) # convert to binary- label 4 = trusted

Test out different kinds of models and find the most effective architectures.|

In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [9]:
print("Training F1 score is:", f1_score(y_train, y_pred_train))
print("Test F1 score is:", f1_score(y_test, y_pred_test))

Training F1 score is: 0.9318290163673925
Test F1 score is: 0.8406827880512091


In [10]:
# save new model
import joblib
joblib.dump(model, './sklearn_models/logisticRegression ' + str(f1_score(y_test, y_pred_test)) + 'tfidf.pkl')

['./sklearn_models/logisticRegression 0.8406827880512091tfidf.pkl']

In [11]:
# load a previous model
import joblib
model = joblib.load("./sklearn_models/logisticRegression.pkl")
y_pred = model.predict(X_train)

FileNotFoundError: [Errno 2] No such file or directory: './sklearn_models/logisticRegression.pkl'

Perform hyperparameter tuning on best 3 models.

In [ ]:
from sklearn.model_selection import GridSearchCV

# or perform hyperparameter tuning
# Create the hyperparameters grid
param_grid = {

}

grid_search = GridSearchCV(())

# Train
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

# Use best model
model = grid_search.best_estimator_